In [25]:
import pandas as pd


fileGenre = open('../data/u.genre', 'r')
genre = {}

count = 0

while True:
    count += 1
    # Get next line from file

    line = fileGenre.readline()

    # if line is empty
    # end of file is reached
    if line.strip() == "":
        break

    #print("{}".format(line.strip()))
    data = line.strip().split('|')

    genre['id'] = int(data[1])
    genre['name'] = data[0]

    print(genre)

fileGenre.close()


{'id': 0, 'name': 'unknown'}
{'id': 1, 'name': 'Action'}
{'id': 2, 'name': 'Adventure'}
{'id': 3, 'name': 'Animation'}
{'id': 4, 'name': "Children's"}
{'id': 5, 'name': 'Comedy'}
{'id': 6, 'name': 'Crime'}
{'id': 7, 'name': 'Documentary'}
{'id': 8, 'name': 'Drama'}
{'id': 9, 'name': 'Fantasy'}
{'id': 10, 'name': 'Film-Noir'}
{'id': 11, 'name': 'Horror'}
{'id': 12, 'name': 'Musical'}
{'id': 13, 'name': 'Mystery'}
{'id': 14, 'name': 'Romance'}
{'id': 15, 'name': 'Sci-Fi'}
{'id': 16, 'name': 'Thriller'}
{'id': 17, 'name': 'War'}
{'id': 18, 'name': 'Western'}
